In [78]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.charts import Histogram, Donut
from bokeh.io import output_notebook
from bokeh.models.formatters import DatetimeTickFormatter as dttf

In [27]:
df=pd.read_json('teamsdone.json')

In [28]:
#gathers the total number of SC2 Tournaments Played
df['totalsc2'] = df.groupby('PlayerId')['PlayerId'].transform('count')

In [29]:
#popped ('DOB' Column for now, as it returned improper values)
df.DOB = pd.to_datetime(df.DOB)
df.EndDate = pd.to_datetime(df.EndDate)

In [30]:
df #changing the nasty index

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,...,PlayerId,Prize,Prize_USD,RankText,TeamPlayers,Team_On,TotalUSDPrize,TournamentName,team_history,totalsc2
0,25.0,kr,KRW,MC,1991-06-17,2016-09-10,0.000905,151,Min Chul,Jang,...,1000,3000000.0,2714.195100,17th-24th,1,CJ Entus,506645.42,GSL Season 2 2016 (Code S),"[{'team': 'CJ Entus', 'history': [146396160000...",108
1,24.0,kr,USD,MC,1991-06-17,2016-02-02,1.000000,151,Min Chul,Jang,...,1000,1000.0,1000.000000,5th-8th,1,Unaffiliated,506645.42,IEM X - Taipei (SC2),"[{'team': 'CJ Entus', 'history': [146396160000...",108
10,23.0,kr,USD,MC,1991-06-17,2014-10-05,1.000000,151,Min Chul,Jang,...,1000,5000.0,5000.000000,5th-8th,1,Unaffiliated,506645.42,WCS Europe 2014 Season 3 Premier,"[{'team': 'CJ Entus', 'history': [146396160000...",108
100,19.0,kr,KRW,MC,1991-06-17,2011-04-09,0.000890,151,Min Chul,Jang,...,1000,10000000.0,8903.530000,3rd-4th,1,Old Generations,506645.42,GSL World Championship Seoul 2011,"[{'team': 'CJ Entus', 'history': [146396160000...",108
1000,17.0,kr,USD,PartinG,1994-08-24,2012-05-27,1.000000,151,Lee Sak,Won,...,1132,1200.0,1200.000000,5th-8th,1,FXOpen e-Sports,355604.37,Red Bull Battlegrounds,"[{'team': 'Flash Wolves', 'history': [14134176...",100
10000,23.0,kr,USD,YongHwa,1992-01-30,2015-07-19,1.000000,151,Yong Hwa,Choi,...,2418,350.0,350.000000,13th-16th,1,MVP,40534.16,IEM X - Shenzhen (StarCraft II),"[{'team': 'MVP', 'history': [1406764800000, 14...",40
10001,23.0,kr,KRW,YongHwa,1992-01-30,2015-04-17,0.000914,151,Yong Hwa,Choi,...,2418,400000.0,365.600000,25th-48th,1,MVP,40534.16,GSL Season 2 2015 (Code A),"[{'team': 'MVP', 'history': [1406764800000, 14...",40
10002,23.0,kr,KRW,YongHwa,1992-01-30,2015-03-22,0.000898,151,Yong Hwa,Choi,...,2418,850000.0,763.201145,17th-24th,1,MVP,40534.16,2015 Global StarCraft II League Season 1,"[{'team': 'MVP', 'history': [1406764800000, 14...",40
10003,23.0,kr,USD,YongHwa,1992-01-30,2015-03-22,1.000000,151,Yong Hwa,Choi,...,2418,15.0,15.000000,3rd-4th,1,MVP,40534.16,OlimoLeague Week #25,"[{'team': 'MVP', 'history': [1406764800000, 14...",40
10004,23.0,kr,USD,YongHwa,1992-01-30,2015-02-26,1.000000,151,Yong Hwa,Choi,...,2418,200.0,200.000000,WIN,7,MVP,40534.16,Showmatch mYinsanity vs. MVP,"[{'team': 'MVP', 'history': [1406764800000, 14...",40


In [31]:
df = df.set_index(np.arange(len(df))) #changing the index to something nice

In [32]:
df

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,...,PlayerId,Prize,Prize_USD,RankText,TeamPlayers,Team_On,TotalUSDPrize,TournamentName,team_history,totalsc2
0,25.0,kr,KRW,MC,1991-06-17,2016-09-10,0.000905,151,Min Chul,Jang,...,1000,3000000.0,2714.195100,17th-24th,1,CJ Entus,506645.42,GSL Season 2 2016 (Code S),"[{'team': 'CJ Entus', 'history': [146396160000...",108
1,24.0,kr,USD,MC,1991-06-17,2016-02-02,1.000000,151,Min Chul,Jang,...,1000,1000.0,1000.000000,5th-8th,1,Unaffiliated,506645.42,IEM X - Taipei (SC2),"[{'team': 'CJ Entus', 'history': [146396160000...",108
2,23.0,kr,USD,MC,1991-06-17,2014-10-05,1.000000,151,Min Chul,Jang,...,1000,5000.0,5000.000000,5th-8th,1,Unaffiliated,506645.42,WCS Europe 2014 Season 3 Premier,"[{'team': 'CJ Entus', 'history': [146396160000...",108
3,19.0,kr,KRW,MC,1991-06-17,2011-04-09,0.000890,151,Min Chul,Jang,...,1000,10000000.0,8903.530000,3rd-4th,1,Old Generations,506645.42,GSL World Championship Seoul 2011,"[{'team': 'CJ Entus', 'history': [146396160000...",108
4,17.0,kr,USD,PartinG,1994-08-24,2012-05-27,1.000000,151,Lee Sak,Won,...,1132,1200.0,1200.000000,5th-8th,1,FXOpen e-Sports,355604.37,Red Bull Battlegrounds,"[{'team': 'Flash Wolves', 'history': [14134176...",100
5,23.0,kr,USD,YongHwa,1992-01-30,2015-07-19,1.000000,151,Yong Hwa,Choi,...,2418,350.0,350.000000,13th-16th,1,MVP,40534.16,IEM X - Shenzhen (StarCraft II),"[{'team': 'MVP', 'history': [1406764800000, 14...",40
6,23.0,kr,KRW,YongHwa,1992-01-30,2015-04-17,0.000914,151,Yong Hwa,Choi,...,2418,400000.0,365.600000,25th-48th,1,MVP,40534.16,GSL Season 2 2015 (Code A),"[{'team': 'MVP', 'history': [1406764800000, 14...",40
7,23.0,kr,KRW,YongHwa,1992-01-30,2015-03-22,0.000898,151,Yong Hwa,Choi,...,2418,850000.0,763.201145,17th-24th,1,MVP,40534.16,2015 Global StarCraft II League Season 1,"[{'team': 'MVP', 'history': [1406764800000, 14...",40
8,23.0,kr,USD,YongHwa,1992-01-30,2015-03-22,1.000000,151,Yong Hwa,Choi,...,2418,15.0,15.000000,3rd-4th,1,MVP,40534.16,OlimoLeague Week #25,"[{'team': 'MVP', 'history': [1406764800000, 14...",40
9,23.0,kr,USD,YongHwa,1992-01-30,2015-02-26,1.000000,151,Yong Hwa,Choi,...,2418,200.0,200.000000,WIN,7,MVP,40534.16,Showmatch mYinsanity vs. MVP,"[{'team': 'MVP', 'history': [1406764800000, 14...",40


In [35]:
output_notebook() #loading bokeh for notebook output, will change to file for later dashboard

Loading BokehJS ...

In [41]:
p = Histogram(df['Age_at_win'].dropna(), title = 'Age of players that cash in tournaments',)

In [43]:
show(p)

In [49]:
p = Histogram(df['totalsc2'].unique(), title='Total Number of Cashes Per Player')

In [50]:
show(p)

In [59]:
df['AveragePrizePerPlayer']=df['TotalUSDPrize']/df['totalsc2'] 

In [60]:
p = Histogram(df['AveragePrizePerPlayer'].unique(), title='Average Prize Per Player')

In [61]:
show(p)

In [62]:
p = Histogram(np.log(df['AveragePrizePerPlayer'].unique()), title='Log Average Prize Per Player')

In [63]:
show(p)

In [66]:
p = Donut(df['CountryCode'], title = "Country of Origin")
show(p)

In [68]:
p = Donut(df['CurrencyCode'], title = 'Currency of Prize Money')
show(p)

In [69]:
pd.set_option('max_columns', 50)

In [70]:
df

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,Note,PlayerId,Prize,Prize_USD,RankText,TeamPlayers,Team_On,TotalUSDPrize,TournamentName,team_history,totalsc2,AveragePrizePerPlayer
0,25.0,kr,KRW,MC,1991-06-17,2016-09-10,0.000905,151,Min Chul,Jang,None,1000,3000000.0,2714.195100,17th-24th,1,CJ Entus,506645.42,GSL Season 2 2016 (Code S),"[{'team': 'CJ Entus', 'history': [146396160000...",108,4691.161296
1,24.0,kr,USD,MC,1991-06-17,2016-02-02,1.000000,151,Min Chul,Jang,None,1000,1000.0,1000.000000,5th-8th,1,Unaffiliated,506645.42,IEM X - Taipei (SC2),"[{'team': 'CJ Entus', 'history': [146396160000...",108,4691.161296
2,23.0,kr,USD,MC,1991-06-17,2014-10-05,1.000000,151,Min Chul,Jang,None,1000,5000.0,5000.000000,5th-8th,1,Unaffiliated,506645.42,WCS Europe 2014 Season 3 Premier,"[{'team': 'CJ Entus', 'history': [146396160000...",108,4691.161296
3,19.0,kr,KRW,MC,1991-06-17,2011-04-09,0.000890,151,Min Chul,Jang,None,1000,10000000.0,8903.530000,3rd-4th,1,Old Generations,506645.42,GSL World Championship Seoul 2011,"[{'team': 'CJ Entus', 'history': [146396160000...",108,4691.161296
4,17.0,kr,USD,PartinG,1994-08-24,2012-05-27,1.000000,151,Lee Sak,Won,None,1132,1200.0,1200.000000,5th-8th,1,FXOpen e-Sports,355604.37,Red Bull Battlegrounds,"[{'team': 'Flash Wolves', 'history': [14134176...",100,3556.043700
5,23.0,kr,USD,YongHwa,1992-01-30,2015-07-19,1.000000,151,Yong Hwa,Choi,None,2418,350.0,350.000000,13th-16th,1,MVP,40534.16,IEM X - Shenzhen (StarCraft II),"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1013.354000
6,23.0,kr,KRW,YongHwa,1992-01-30,2015-04-17,0.000914,151,Yong Hwa,Choi,None,2418,400000.0,365.600000,25th-48th,1,MVP,40534.16,GSL Season 2 2015 (Code A),"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1013.354000
7,23.0,kr,KRW,YongHwa,1992-01-30,2015-03-22,0.000898,151,Yong Hwa,Choi,None,2418,850000.0,763.201145,17th-24th,1,MVP,40534.16,2015 Global StarCraft II League Season 1,"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1013.354000
8,23.0,kr,USD,YongHwa,1992-01-30,2015-03-22,1.000000,151,Yong Hwa,Choi,None,2418,15.0,15.000000,3rd-4th,1,MVP,40534.16,OlimoLeague Week #25,"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1013.354000
9,23.0,kr,USD,YongHwa,1992-01-30,2015-02-26,1.000000,151,Yong Hwa,Choi,None,2418,200.0,200.000000,WIN,7,MVP,40534.16,Showmatch mYinsanity vs. MVP,"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1013.354000


In [80]:
p = figure()
p.circle(x = df['EndDate'], y =df['Prize_USD'])
p.xaxis.formatter = dttf.months

ValueError: expected an instance of type TickFormatter, got List(String) of type BasicPropertyDescriptor

In [77]:
show(p)